RMinimum : Full - Test - Case: $k(n) = n^{\varepsilon}$ - $\varepsilon$ fix

In [10]:
import math
import random
import queue

Testfälle : $k(n) = n^{\varepsilon}$, $X_{n_0} = [0, \cdots, n_0-1], \cdots, X_{n_1}=[0, \cdots, n_1-1]$

In [16]:
# User input : Range [n0, n1]
n0, n1 = 6, 11
eps = 0.5

# Automatic generation:
lst_X, lst_n = [], []
for n in range(n0, n1 + 1, 2):
    lst_X.append([i for i in range(2**n)])
    lst_n.append(2**n)
lst_k = [int(len(x)**eps) for x in lst_X]

# Show Testcase
print('')
print('Input tuples : (n, k)')
print('=====================')
for i in range(len(lst_X)):
    print(i+1, ':', '(', str(len(lst_X[i])).ljust(5), ',', str(lst_k[i]).ljust(3), ')')


Input tuples : (n, k)
1 : ( 64    , 8   )
2 : ( 256   , 16  )
3 : ( 1024  , 32  )


Algorithmus : Full

In [13]:
def rminimum(X,k, cnt = []):
    k = int(k)
    n = len(X)
    if cnt == []:
        cnt = [0 for _ in range(len(X))]

    if len(X) == 3:
        if X[0] < X[1]:
            cnt[X[0]] += 2
            cnt[X[1]] += 1
            cnt[X[2]] += 1

            if X[0] < X[2]:
                X = X[0]
            else:
                X = X[2]

        else:
            cnt[X[0]] += 1
            cnt[X[1]] += 2
            cnt[X[2]] += 1

            if X[1] < X[2]:
                X = X[1]
            else:
                X = X[2]

        return cnt

    W, L, cnt = RMinimum_step1(X, cnt)
    minele, cnt = RMinimum_step2(L, k, cnt)
    res3, cnt = RMinimum_step3(W, k, minele, cnt)
    res4, cnt = RMinimum_step4(res3, k, n, cnt)

    return cnt

# ==================================================
def RMinimum_step1(lst, cnt):

    random.shuffle(lst)

    W = [0 for _ in range(len(lst) // 2)]
    L = [0 for _ in range(len(lst) // 2)]

    for i in range(len(lst) // 2):
        if lst[2 * i] > lst[2 * i + 1]:
            W[i] = lst[2 * i + 1]
            L[i] = lst[2 * i]
        else:
            W[i] = lst[2 * i]
            L[i] = lst[2 * i + 1]
        cnt[lst[2 * i + 1]] += 1
        cnt[lst[2 * i]] += 1

    return W, L, cnt

# ==================================================
def RMinimum_step2(L, k, cnt):

    random.shuffle(L)
    res = [L[i * k:(i + 1) * k] for i in range((len(L) + k - 1) // k)]
    minele = [0 for _ in range(len(res))]

    var = list(res)
    for i in range(len(var)):
        q = queue.Queue()

        for item in var[i]:
            q.put(item)

        while q.qsize() > 1:
            a = q.get()
            b = q.get()

            if a < b:
                q.put(a)
            else:
                q.put(b)
            cnt[a] += 1
            cnt[b] += 1
        minele[i] = q.get()

    return minele, cnt

# ==================================================
def RMinimum_step3(lst, k, minele, cnt):

    random.shuffle(lst)
    var = [lst[i * k:(i + 1) * k] for i in range((len(lst) + k - 1) // k)]
    res = [0 for _ in range(len(var))]

    for i in range(len(var)):
        res[i] = [elem for elem in var[i] if elem < minele[i]]
        cnt[minele[i]] += len(var[i])
        for elem in var[i]:
            cnt[elem] += 1

    res = [item for sublist in res for item in sublist]

    return res, cnt

# ==================================================
def RMinimum_step4(newW, k, n, cnt):

    if len(newW) <= (math.log(n)/math.log(2))**2:
        q = queue.Queue()

        var = list(newW)
        for item in var:
            q.put(item)
        while q.qsize() > 1:
            a = q.get()
            b = q.get()

            if a < b:
                q.put(a)
            else:
                q.put(b)

            cnt[a] += 1
            cnt[b] += 1
        res = q.get()

    else:
        res = rminimum(newW,k, cnt)

    return res, cnt

# ==================================================
# Testfall
lst_cnt = []
for i in range(len(lst_X)):
    lst_cnt.append(rminimum(lst_X[i], lst_k[i]))

Resultat : $\mathbb{E}[f_{min}] = \mathcal{O}(\varepsilon^{-1}\log\log(n))$, $\mathbb{E}[f_{rem}]=\mathcal{O}(n^{\varepsilon})$, $\mathbb{E}[work]=\mathcal{O}(n)$.

In [14]:
def test(lst_n, lst_k, eps, lst_cnt, n0, n1):
          
    print('')
    print('E[f_min] = O(eps^(-1) * loglog(n))   |   E[f_rem] = O(n^eps)   |   E[work] = O(n)')

    for i in range(len(lst_n)):
        n = lst_n[i]
        f_min = lst_cnt[i][0]
        f_rem = max(lst_cnt[i][1:])
        work = int(sum(lst_cnt[i])/2)
        E_min = round((eps**(-1) * (math.log(n)/math.log(2)) / (math.log(math.log(n)/math.log(2))/math.log(2))), 2)
        E_rem = round(n**eps, 2)
        E_work = n
        
        print('')
        print('Testfall n | k | eps :', lst_n[i], '|', lst_k[i], '|', eps)
        print('==========================================')
        print('f_min | E[f_min] | Diff :', f_min, '|', E_min, '|', round(abs(f_min - E_min), 2))
        print('------------------------------------------')    
        print('f_rem | E[f_rem] | Diff :', f_rem, '|', E_rem, '|', round(abs(f_rem - E_rem), 2))
        print('------------------------------------------')
        print('Work  | E[work]  | Diff :', work, '|', E_work, '|', round(abs(work - E_work), 2))
        print('==========================================')
    return

# Testfall
test(lst_n, lst_k, eps, lst_cnt, n0, n1)


E[f_min] = O(eps^(-1) * loglog(n))   |   E[f_rem] = O(n^eps)   |   E[work] = O(n)

Testfall n | k | eps : 64 | 8 | 0.5
f_min | E[f_min] | Diff : 5 | 4.64 | 0.36
------------------------------------------
f_rem | E[f_rem] | Diff : 12 | 8.0 | 4.0
------------------------------------------
Work  | E[work]  | Diff : 106 | 64 | 42

Testfall n | k | eps : 256 | 16 | 0.5
f_min | E[f_min] | Diff : 8 | 5.33 | 2.67
------------------------------------------
f_rem | E[f_rem] | Diff : 21 | 16.0 | 5.0
------------------------------------------
Work  | E[work]  | Diff : 414 | 256 | 158

Testfall n | k | eps : 1024 | 32 | 0.5
f_min | E[f_min] | Diff : 8 | 6.02 | 1.98
------------------------------------------
f_rem | E[f_rem] | Diff : 40 | 32.0 | 8.0
------------------------------------------
Work  | E[work]  | Diff : 1722 | 1024 | 698
